In [42]:
#import necessary packages
import pandas as pd
import json
from pandas.io.json import json_normalize
import matplotlib.pyplot as plt

In [43]:
#load world_bank_projects.json file and read into a dataframe using json_normalize since the file has nested dictionaries
with open('world_bank_projects.json') as file:
    wb=json.load(file)
wb=json_normalize(wb)
#check data type of wb
print(type(wb))

<class 'pandas.core.frame.DataFrame'>


In [44]:
#check columns of data frame
wb.columns

Index(['_id.$oid', 'approvalfy', 'board_approval_month', 'boardapprovaldate',
       'borrower', 'closingdate', 'country_namecode', 'countrycode',
       'countryname', 'countryshortname', 'docty', 'envassesmentcategorycode',
       'grantamt', 'ibrdcommamt', 'id', 'idacommamt', 'impagency',
       'lendinginstr', 'lendinginstrtype', 'lendprojectcost',
       'majorsector_percent', 'mjsector_namecode', 'mjtheme',
       'mjtheme_namecode', 'mjthemecode', 'prodline', 'prodlinetext',
       'productlinetype', 'project_abstract.cdata', 'project_name',
       'projectdocs', 'projectfinancialtype', 'projectstatusdisplay',
       'regionname', 'sector', 'sector1.Name', 'sector1.Percent',
       'sector2.Name', 'sector2.Percent', 'sector3.Name', 'sector3.Percent',
       'sector4.Name', 'sector4.Percent', 'sector_namecode', 'sectorcode',
       'source', 'status', 'supplementprojectflg', 'theme1.Name',
       'theme1.Percent', 'theme_namecode', 'themecode', 'totalamt',
       'totalcommamt', 

In [45]:
#aim 1 is to find the top 10 countries with most projects, first check for integrity of data
wb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 55 columns):
_id.$oid                    500 non-null object
approvalfy                  500 non-null object
board_approval_month        500 non-null object
boardapprovaldate           500 non-null object
borrower                    485 non-null object
closingdate                 370 non-null object
country_namecode            500 non-null object
countrycode                 500 non-null object
countryname                 500 non-null object
countryshortname            500 non-null object
docty                       446 non-null object
envassesmentcategorycode    430 non-null object
grantamt                    500 non-null int64
ibrdcommamt                 500 non-null int64
id                          500 non-null object
idacommamt                  500 non-null int64
impagency                   472 non-null object
lendinginstr                495 non-null object
lendinginstrtype            495 no

In [46]:
#since both 'countryname' and 'projectname' have 500 non-null entries, we can just do value counts of 'countryname' (or code)
top_10_countries=wb['countryname'].value_counts().head(10)
print(top_10_countries)

People's Republic of China         19
Republic of Indonesia              19
Socialist Republic of Vietnam      17
Republic of India                  16
Republic of Yemen                  13
People's Republic of Bangladesh    12
Kingdom of Morocco                 12
Nepal                              12
Africa                             11
Republic of Mozambique             11
Name: countryname, dtype: int64


In [47]:
#to find the top 10 projects, use the 'mjtheme_namecode' column, which also has 500 non null values
#'mjtheme_namecode' columns has lists of dictionaries, convert to dataframes for counting
themes=wb['mjtheme_namecode'].apply(lambda x: pd.DataFrame(x, columns=['code', 'name']))
#concat and make a new data frame with all 'code' and 'name' entries from the 'mjtheme_namecode' column
all_themes=pd.concat(list(themes))
#print firts 10 records to check data
all_themes.head(10)

,code,name
0,8,Human development
1,11,
0,1,Economic management
1,6,Social protection and risk management
0,5,Trade and integration
1,2,Public sector governance
2,11,Environment and natural resources management
3,6,Social protection and risk management
0,7,Social dev/gender/inclusion
1,7,Social dev/gender/inclusion


In [48]:
#noticed some emtry entries in the 'name' column. Need to drop those when counting
space_filter=(all_themes['name']!='')
all_themes_tidy=all_themes[space_filter]
#check to see if now 'name' column has no empty entries. The following code should return 0
(all_themes_tidy['name']=='').sum()

0

In [49]:
#check data by printing first 10 records
all_themes_tidy.head(10)

,code,name
0,8,Human development
0,1,Economic management
1,6,Social protection and risk management
0,5,Trade and integration
1,2,Public sector governance
2,11,Environment and natural resources management
3,6,Social protection and risk management
0,7,Social dev/gender/inclusion
1,7,Social dev/gender/inclusion
0,5,Trade and integration


In [50]:
#now we can count top 10 project themes by doing value count on 'name' column
top_10_projects=all_themes_tidy['name'].value_counts().head(10)
print(top_10_projects)

Environment and natural resources management    223
Rural development                               202
Human development                               197
Public sector governance                        184
Social protection and risk management           158
Financial and private sector development        130
Social dev/gender/inclusion                     119
Trade and integration                            72
Urban development                                47
Economic management                              33
Name: name, dtype: int64


In [51]:
#now we need to fill in those missing values for the 'name' in the original data frame
#since all the 'codes' should have a proper mapping in the new data frame after dropping empty entries, we can contruct a new dictionary 
#with all unique codes and names to map to original data frame using a function
#first drop all duplicates in the new data frame to get unique entries
all_themes_unique=all_themes_tidy.drop_duplicates()
#compare before and after dropping duplicate values
print('before ', len(all_themes_tidy), ';', 'after ', len(all_themes_unique))

before  1377 ; after  11


In [52]:
#so there are only 11 codes, each corresponding to a unique name
#construct a dictionary to reflect the mapping
code_to_name=dict(zip(all_themes_unique['code'], all_themes_unique['name']))
#check to see if the dictionary is correct
print(code_to_name)

{'8': 'Human development', '1': 'Economic management', '6': 'Social protection and risk management', '5': 'Trade and integration', '2': 'Public sector governance', '11': 'Environment and natural resources management', '7': 'Social dev/gender/inclusion', '4': 'Financial and private sector development', '10': 'Rural development', '9': 'Urban development', '3': 'Rule of law'}


In [53]:
#now we have codes properly mapped to names, we can apply a function to fill the missing values
def replace_missing_name(list):
    """The list is a list containing dictionaries where some keys are missing values.
       The missing values will be replaced by mappings from another dictionary"""
    for d in list:
        if not d['name']:
            d['name']=code_to_name[d['code']]
    return(list)

#apply this function to original data frame's mjtheme_namecode column to fill missing names
#first check to see that the first entry in the original data frame has a missing name
print(wb['mjtheme_namecode'][0])
#so for 'code' 11 the name is missing
#let's make a copy of the original data frame to test the function
wb_copy=wb.copy()
wb_copy['mjtheme_namecode'].apply(replace_missing_name)
#check first record of 'mjtheme_namecode column again
print(wb_copy['mjtheme_namecode'][0])
#looks like it worked


[{'code': '8', 'name': 'Human development'}, {'code': '11', 'name': ''}]
[{'code': '8', 'name': 'Human development'}, {'code': '11', 'name': 'Environment and natural resources management'}]


In [62]:
#print top 10 countries with most projects
print(top_10_countries)

People's Republic of China         19
Republic of Indonesia              19
Socialist Republic of Vietnam      17
Republic of India                  16
Republic of Yemen                  13
People's Republic of Bangladesh    12
Kingdom of Morocco                 12
Nepal                              12
Africa                             11
Republic of Mozambique             11
Name: countryname, dtype: int64


In [63]:
#print top 10 major project themes
print(top_10_projects)

Environment and natural resources management    223
Rural development                               202
Human development                               197
Public sector governance                        184
Social protection and risk management           158
Financial and private sector development        130
Social dev/gender/inclusion                     119
Trade and integration                            72
Urban development                                47
Economic management                              33
Name: name, dtype: int64


In [69]:
#the modified data frame without misisng names for the 'mjtheme_namecode' column
pd.set_option('max_colwidth' ,150)
print(wb_copy['mjtheme_namecode'].head(10))

0                                     [{'code': '8', 'name': 'Human development'}, {'code': '11', 'name': 'Environment and natural resources management'}]
1                                           [{'code': '1', 'name': 'Economic management'}, {'code': '6', 'name': 'Social protection and risk management'}]
2    [{'code': '5', 'name': 'Trade and integration'}, {'code': '2', 'name': 'Public sector governance'}, {'code': '11', 'name': 'Environment and natura...
3                                             [{'code': '7', 'name': 'Social dev/gender/inclusion'}, {'code': '7', 'name': 'Social dev/gender/inclusion'}]
4                                      [{'code': '5', 'name': 'Trade and integration'}, {'code': '4', 'name': 'Financial and private sector development'}]
5                         [{'code': '6', 'name': 'Social protection and risk management'}, {'code': '6', 'name': 'Social protection and risk management'}]
6                                   [{'code': '2', 'name': 'Public sec